# Object Detection using TAO RetinaNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

## Sample prediction of RetinaNet
<img align="center" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/common/sample.jpg?raw=true" width="960">

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 RetinaNet model on the KITTI dataset
* Prune the trained retinanet model
* Retrain the pruned model to recover lost accuracy
* Export the pruned model
* Quantize the pruned model using QAT
* Run Inference on the trained model
* Export the pruned, quantized and retrained model to a .onnx file for deployment to DeepStream
* Run inference on the exported .onnx model to verify deployment using TensorRT

At the end of this notebook, you will have generated a trained and optimized `retinanet` model
which you may deploy via [Triton](https://github.com/NVIDIA-AI-IOT/tao-toolkit-triton-apps)
or [DeepStream](https://developer.nvidia.com/deepstream-sdk).

## Table of Contents

This notebook shows an example usecase of RetinaNet object detection using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Install the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2) <br>
    2.1 [Download the dataset](#head-2-1) <br>
    2.2 [Validate the downloaded dataset](#head-2-2) <br>
    2.3 [Generate tfrecords from kitti format dataset](#head-2-3) <br>
    2.4 [Download pre-trained model](#head-2-4) <br>
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Prune trained models](#head-6)
7. [Retrain pruned models](#head-7)
8. [Evaluate retrained model](#head-8)
9. [Visualize inferences](#head-9)
10. [Model Export](#head-10)
11. [Verify deployed model](#head-11)
12. [QAT Workflow](#head-12)

## 0. Set up env variables <a class="anchor" id="head-0"></a>

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/retinanet`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/retinanet
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/retinanet

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/retinanet
os.environ["LOCAL_PROJECT_DIR"] = FIXME
os.environ["LOCAL_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data")
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "retinanet")

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tao-experiments/retinanet/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/text/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
```

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Install the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

After setting up your virtual environment with the above requirements, install TAO pip package

In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info

## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) and labels (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) to $DATA_DOWNLOAD_DIR.

### 2.1. Download the dataset <a class="anchor" id="head-2-1"></a>
Once you have gotten the download links in your email, please populate them in place of the `KITTI_IMAGES_DOWNLOAD_URL` and the `KITTI_LABELS_DOWNLOAD_URL`. This next cell, will download the data and place in `$LOCAL_DATA_DIR`

In [ ]:
# Create local dir
!mkdir -p $LOCAL_DATA_DIR
!mkdir -p $LOCAL_EXPERIMENT_DIR

In [ ]:
import os
os.environ["URL_IMAGES"]=KITTI_IMAGES_DOWNLOAD_URL
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then wget $URL_IMAGES -O $LOCAL_DATA_DIR/data_object_image_2.zip; else echo "image archive already downloaded"; fi 
os.environ["URL_LABELS"]=KITTI_LABELS_DOWNLOAD_URL
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then wget $URL_LABELS -O $LOCAL_DATA_DIR/data_object_label_2.zip; else echo "label archive already downloaded"; fi 

### 2.2. Validate the downloaded dataset <a class="anchor" id="head-2-2"></a>

In [ ]:
# Check the dataset is present
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

In [ ]:
# This may take a while: verify integrity of zip files 
!sha256sum $LOCAL_DATA_DIR/data_object_image_2.zip | cut -d ' ' -f 1 | grep -xq '^351c5a2aa0cd9238b50174a3a62b846bc5855da256b82a196431d60ff8d43617$' ; \
if test $? -eq 0; then echo "images OK"; else echo "images corrupt, redownload!" && rm -f $LOCAL_DATA_DIR/data_object_image_2.zip; fi 
!sha256sum $LOCAL_DATA_DIR/data_object_label_2.zip | cut -d ' ' -f 1 | grep -xq '^4efc76220d867e1c31bb980bbf8cbc02599f02a9cb4350effa98dbb04aaed880$' ; \
if test $? -eq 0; then echo "labels OK"; else echo "labels corrupt, redownload!" && rm -f $LOCAL_DATA_DIR/data_object_label_2.zip; fi 

In [ ]:
# unpack 
!unzip -u $LOCAL_DATA_DIR/data_object_image_2.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/data_object_label_2.zip -d $LOCAL_DATA_DIR

In [ ]:
# verify
!ls -l $LOCAL_DATA_DIR

Additionally, if you have your own dataset already in a volume (or folder), you can mount the volume on `LOCAL_DATA_DIR` (or create a soft link). Below shows an example:
```bash
# if your dataset is in /dev/sdc1
mount /dev/sdc1 $LOCAL_DATA_DIR

# if your dataset is in folder /var/dataset
ln -sf /var/dataset $LOCAL_DATA_DIR
```

In [ ]:
# Directory where splitted dataset will be located
!mkdir -p $LOCAL_DATA_DIR/kitti_split
# Generate val dataset out of training dataset
!python3 ../ssd/generate_split.py --input_image_dir=$LOCAL_DATA_DIR/training/image_2 \
                                  --input_label_dir=$LOCAL_DATA_DIR/training/label_2 \
                                  --output_dir=$LOCAL_DATA_DIR/kitti_split

### 2.3 Generate tfrecords from kitti format dataset <a class="anchor" id="head-2-3"></a>

- Update the tfrecords spec file to take in your kitti format dataset
- Create the tfrecords using the `dataset_convert`

*Note: TFRecords only need to be generated for the training set once.*

In [ ]:
print("TFRecords conversion spec file:")
!cat $LOCAL_SPECS_DIR/retinanet_tfrecords_kitti_train.txt

In [ ]:
# Creating a new directory for the output tfrecords dump.
print("Converting the training set to TFRecords.")
!tao model retinanet dataset_convert \
               -d $SPECS_DIR/retinanet_tfrecords_kitti_train.txt \
               -o $DATA_DOWNLOAD_DIR/retinanet/tfrecords/kitti_train \
               -r $USER_EXPERIMENT_DIR/

In [ ]:
!ls -rlt $LOCAL_DATA_DIR/retinanet/tfrecords/

### 2.4 Download pre-trained model <a class="anchor" id="head-2-4"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to [ngc.nvidia.com](ngc.nvidia.com) and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm -f $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list nvidia/tao/pretrained_object_detection:*

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/

In [ ]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet18 \
                    --dest $LOCAL_EXPERIMENT_DIR/pretrained_resnet18

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18

## 3. Provide training specification <a class="anchor" id="head-3"></a>
* Pre-trained models
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.
* *Note* that in the provided spec file, anchor boxes are set to have 3 aspect ratios (`aspect_ratios_global`) and 3 anchor sizes(`n_anchor_levels`) per feature map cell.
* *Note* that the provided spec file uses `batch_size_per_gpu: 24`, which assumes at least 16G GPU memory. If you need to adjust batch size, please adjust the learning rate accordingly.

In [ ]:
!cat $LOCAL_SPECS_DIR/retinanet_train_resnet18_kitti.txt

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned

In [ ]:
!sed -i "s|YOUR_PRETRAINED_MODEL|$USER_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18/resnet_18.hdf5|g" $LOCAL_SPECS_DIR/retinanet_train_resnet18_kitti.txt
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!tao model retinanet train -e $SPECS_DIR/retinanet_train_resnet18_kitti.txt \
                     -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
                     --gpus 1

In [ ]:
print("To resume training from a checkpoint, you need to update the spec file.")
print("use resume_model_path instead of pretrain_model_path with the checkpoint path you wish to resume from.")

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights

In [ ]:
# You can check the evaluation stats in the csv file and pick the model with highest val accuracy.
!cat $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/retinanet_training_log_resnet18.csv
%set_env EPOCH=100

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

In [ ]:
!tao model retinanet evaluate -e $SPECS_DIR/retinanet_train_resnet18_kitti.txt \
                        -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/retinanet_resnet18_epoch_$EPOCH.hdf5

## 6. Prune trained models <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion (`Only for resnets as they have element wise operations or MobileNets.`)
* Threshold for pruning.
* Output directory to store the model

Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold value depends on the dataset and the model. `0.4` in the block below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
# Create an output directory to save the pruned model.
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned

In [ ]:
!tao model retinanet prune -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/retinanet_resnet18_epoch_$EPOCH.hdf5 \
               -o $USER_EXPERIMENT_DIR/experiment_dir_pruned/retinanet_resnet18_pruned.hdf5 \
               -pth 0.4

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/

## 7. Retrain pruned models <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification
* WARNING: training will take several hours or one day to complete

In [ ]:
# Printing the retrain spec file. 
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!sed -i "s|YOUR_PRETRAINED_MODEL|$USER_EXPERIMENT_DIR/experiment_dir_pruned/retinanet_resnet18_pruned.hdf5|g" $LOCAL_SPECS_DIR/retinanet_retrain_resnet18_kitti.txt
!cat $LOCAL_SPECS_DIR/retinanet_retrain_resnet18_kitti.txt

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain

In [ ]:
# Retraining using the pruned model as pretrained weights.
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!tao model retinanet train --gpus 1 \
                     -e $SPECS_DIR/retinanet_retrain_resnet18_kitti.txt \
                     -r $USER_EXPERIMENT_DIR/experiment_dir_retrain

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights

In [ ]:
# You can check the evaluation stats in the csv file and pick the model with highest val accuracy.
!cat $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/retinanet_training_log_resnet18.csv
%set_env EPOCH=100

## 8. Evaluate retrained model <a class="anchor" id="head-8"></a>

In [ ]:
!tao model retinanet evaluate -e $SPECS_DIR/retinanet_retrain_resnet18_kitti.txt \
                        -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/retinanet_resnet18_epoch_$EPOCH.hdf5

## 9. Visualize inferences <a class="anchor" id="head-9"></a>
In this section, we run the tlt-infer tool to generate inferences on the trained models and visualize the results.

In [ ]:
# Copy some test images
!mkdir -p $LOCAL_DATA_DIR/test_samples
!cp $LOCAL_DATA_DIR/testing/image_2/00000* $LOCAL_DATA_DIR/test_samples

In [ ]:
# Running inference for detection on n images
!tao model retinanet inference -i $DATA_DOWNLOAD_DIR/test_samples \
                         -e $SPECS_DIR/retinanet_retrain_resnet18_kitti.txt \
                         -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/retinanet_resnet18_epoch_$EPOCH.hdf5 \
                         -r $USER_EXPERIMENT_DIR

The `inference` tool produces two outputs. 
1. Overlain images in `$LOCAL_EXPERIMENT_DIR/images_annotated`
2. Frame by frame bbox labels in kitti format located in `$LOCAL_EXPERIMENT_DIR/labels`

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=1, num_images=1):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'images_annotated' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 10. Model Export <a class="anchor" id="head-10"></a>

In [ ]:
# tao <task> export will fail if .onnx already exists. So we clear the export folder before tao <task> export
!rm -rf $LOCAL_EXPERIMENT_DIR/export
# Generate .onnx file using tao container
!mkdir -p $LOCAL_EXPERIMENT_DIR/export 
!tao model retinanet export -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/retinanet_resnet18_epoch_$EPOCH.hdf5 \
                      -e $SPECS_DIR/retinanet_retrain_resnet18_kitti.txt \
                      --gen_ds_config

# This will create a .onnx file at:
# $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights/retinanet_resnet18_epoch_$EPOCH.onnx

Using the `tao deploy` container, you can generate a TensorRT engine and verify the correctness of the generated through evaluate and inference. 

The `tao deploy` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please run `tao deploy` command which will instantiate a deploy container, with the exported `.onnx` file on your target device. The `tao deploy` container only works for x86, with discrete NVIDIA GPU's. 

For the jetson devices, please download the tao-converter for jetson from the dev zone link [here](https://developer.nvidia.com/tao-converter). 

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.onnx` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [ ]:
!tao deploy retinanet gen_trt_engine -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/retinanet_resnet18_epoch_$EPOCH.onnx \
                      -e $SPECS_DIR/retinanet_retrain_resnet18_kitti.txt \
                      --engine_file $USER_EXPERIMENT_DIR/export/retinanet_resnet18_epoch_$EPOCH.engine \
                      --batch_size 1 \
                      --data_type fp32 \
                      --results_dir $USER_EXPERIMENT_DIR/export

# This will create a TRT engine file at:
# $LOCAL_EXPERIMENT_DIR/export/retinanet_resnet18_epoch_$EPOCH.engine

In [ ]:
# Export in FP16 mode. 
!tao deploy retinanet gen_trt_engine -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/retinanet_resnet18_epoch_$EPOCH.onnx \
                      -e $SPECS_DIR/retinanet_retrain_resnet18_kitti.txt \
                      --engine_file $USER_EXPERIMENT_DIR/export/retinanet_resnet18_epoch_$EPOCH.engine.fp16 \
                      --batch_size 1 \
                      --data_type fp16 \
                      --results_dir $USER_EXPERIMENT_DIR/export

In [ ]:
# Export in INT8 mode(generate calibration cache file).
# This will generate an .onnx file and an INT8 calibration file.
!tao deploy retinanet gen_trt_engine -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/retinanet_resnet18_epoch_${EPOCH}.onnx \
                                     -e $SPECS_DIR/retinanet_retrain_resnet18_kitti.txt \
                                     --cal_image_dir $DATA_DOWNLOAD_DIR/training/image_2 \
                                     --data_type int8 \
                                     --batch_size 1 \
                                     --batches 10 \
                                     --engine_file $USER_EXPERIMENT_DIR/export/retinanet_resnet18_epoch_$EPOCH.engine.int8 \
                                     --cal_cache_file $USER_EXPERIMENT_DIR/export/cal.bin \
                                     --cal_data_file $USER_EXPERIMENT_DIR/export/cal.tensorfile \
                                     --results_dir $USER_EXPERIMENT_DIR/export

`Note:` In this example, for ease of execution we restrict the number of calibrating batches to 10. TAO Toolkit recommends the use of at least 10% of the training dataset for int8 calibration.

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $LOCAL_EXPERIMENT_DIR/export

## 11. Verify the deployed model <a class="anchor" id="head-11"></a>
Verify the converted engine by visualizing TensorRT inferences.

In [ ]:
# Infer using TensorRT engine
# Please feel free to use any batch size that the GPU may be able to fit. 
# The engine batch size once created, cannot be alterred. So if you wish to run with a different batch-size,
# please re-run tao deploy <task> gen_trt_engine with the new batch-size.
!tao deploy retinanet inference -m $USER_EXPERIMENT_DIR/export/retinanet_resnet18_epoch_$EPOCH.engine \
                                -e $SPECS_DIR/retinanet_retrain_resnet18_kitti.txt \
                                -i $DATA_DOWNLOAD_DIR/test_samples \
                                -r $USER_EXPERIMENT_DIR/export/trt_infer_results \
                                -t 0.4

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'export/trt_infer_results/images_annotated' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 12. QAT workflow <a class="anchor" id="head-12"></a>
In this section, we will explore the typical Quantization-Aware Training(QAT) workflow with TAO. QAT workflow is almost the same as non-QAT workflow except for two major differences:
1. set `enable_qat` to `True` in training and retraining spec files to enable the QAT for training/retraining
2. when doing export in INT8 mode, the calibration json file that stores the scales used during QAT is extracted during `tao <task> export`. The .onnx file and calibration json file are used to generate engine file through tao deploy

### 12.1 Training

In [ ]:
!sed -i "s|YOUR_PRETRAINED_MODEL|$USER_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18/resnet_18.hdf5|g" $LOCAL_SPECS_DIR/retinanet_train_resnet18_kitti_qat.txt
!cat $LOCAL_SPECS_DIR/retinanet_train_resnet18_kitti_qat.txt

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned_qat
!tao model retinanet train -e $SPECS_DIR/retinanet_train_resnet18_kitti_qat.txt \
                     -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned_qat \
                     --gpus 1

### 12.2 Evaluation

In [ ]:
%set_env EPOCH=100
!tao model retinanet evaluate -e $SPECS_DIR/retinanet_train_resnet18_kitti_qat.txt \
                        -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned_qat/weights/retinanet_resnet18_epoch_$EPOCH.hdf5

### 12.3 Pruning

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned_qat
print("To prune a QAT model:")
!tao model retinanet prune -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned_qat/weights/retinanet_resnet18_epoch_$EPOCH.hdf5 \
                     -o $USER_EXPERIMENT_DIR/experiment_dir_pruned_qat/retinanet_resnet18_pruned.hdf5 \
                     -pth 0.4

### 12.4 Retraining

In [ ]:
!sed -i "s|YOUR_PRETRAINED_MODEL|$USER_EXPERIMENT_DIR/experiment_dir_pruned_qat/retinanet_resnet18_pruned.hdf5|g" $LOCAL_SPECS_DIR/retinanet_retrain_resnet18_kitti_qat.txt
!cat $LOCAL_SPECS_DIR/retinanet_retrain_resnet18_kitti_qat.txt

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat
!tao model retinanet train --gpus 1 \
                     -e $SPECS_DIR/retinanet_retrain_resnet18_kitti_qat.txt \
                     -r $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat

### 12.5 Evaluation of retrained model

In [ ]:
!tao model retinanet evaluate -e $SPECS_DIR/retinanet_retrain_resnet18_kitti_qat.txt \
                        -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/retinanet_resnet18_epoch_$EPOCH.hdf5

### 12.6 Inference of retrained model

In [ ]:
# Running inference for detection on n images
!tao model retinanet inference -i $DATA_DOWNLOAD_DIR/test_samples \
                         -e $SPECS_DIR/retinanet_retrain_resnet18_kitti_qat.txt \
                         -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/retinanet_resnet18_epoch_$EPOCH.hdf5 \
                         -r $USER_EXPERIMENT_DIR/

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'images_annotated' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

### 12.7 Deployment of QAT model

If you train a QAT model, you may only export in INT8 mode using following code block. This generates an onnx file and the corresponding calibration json file that stores scales used during QAT. You can either use the onnx file and calibration json file to generate int8 engine through tao deploy or DeepStream for FP32 or FP16 mode. But please note this gives sub-optimal results. If you want to deploy in FP32 or FP16, you should disable QAT in training.

In [ ]:
# Calibration JSON file is required for INT8 engine generation from tao deploy
!rm -f $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/retinanet_resnet18_epoch_${EPOCH}.onnx
!mkdir -p $LOCAL_EXPERIMENT_DIR/export_qat
!tao model retinanet export -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/retinanet_resnet18_epoch_$EPOCH.hdf5  \
                      -o $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/retinanet_resnet18_epoch_$EPOCH.onnx \
                      -e $SPECS_DIR/retinanet_retrain_resnet18_kitti_qat.txt \
                      --cal_json_file $USER_EXPERIMENT_DIR/export_qat/cal.json \
                      --gen_ds_config

In [ ]:
# Export in INT8 mode(generate calibration cache file).
# This will generate an .onnx file and an INT8 calibration file.
!tao deploy retinanet gen_trt_engine -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/retinanet_resnet18_epoch_${EPOCH}.onnx \
                                     -e $SPECS_DIR/retinanet_retrain_resnet18_kitti_qat.txt \
                                     --data_type int8 \
                                     --batch_size 1 \
                                     --batches 10 \
                                     --engine_file $USER_EXPERIMENT_DIR/export_qat/retinanet_resnet18_epoch_$EPOCH.engine.qat \
                                     --cal_json_file $USER_EXPERIMENT_DIR/export_qat/cal.json \
                                     --cal_cache_file $USER_EXPERIMENT_DIR/export_qat/cal.bin \
                                     --results_dir $USER_EXPERIMENT_DIR/export_qat

### 12.8 Verify TensorRT engine

In [ ]:
# Infer using TensorRT engine
# Please feel free to use any batch size that the GPU may be able to fit. 
# The engine batch size once created, cannot be alterred. So if you wish to run with a different batch-size,
# please re-run tao deploy <task> gen_trt_engine with the new batch-size.
!tao deploy retinanet inference -m $USER_EXPERIMENT_DIR/export_qat/retinanet_resnet18_epoch_$EPOCH.engine.qat \
                                -e $SPECS_DIR/retinanet_retrain_resnet18_kitti_qat.txt \
                                -i $DATA_DOWNLOAD_DIR/test_samples \
                                -r $USER_EXPERIMENT_DIR/export_qat/trt_infer_results \
                                -t 0.4